# Segmentação de sinais

A segmentação é feita de forma temporal, de acordo com classificação de período da crise.


São atribuidos os seguintes rótulos

- não-ictal (NI): sem crise;
- ictal (I): durante a crise;

In [73]:
import pandas as pd

In [74]:
infos_path = '/home/davi/Documentos/doutorado_ppgee_v2/data/siena_infos.json'
path_data = '/media/davi/6A81-05CF/physionet.org/files/siena-scalp-eeg/1.0.0/PN00/'

infos = pd.read_json(infos_path)

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Doutorado').getOrCreate()

In [76]:
# Inserindo labels

from pyspark.sql.functions import expr
from pyspark.sql import DataFrame

def set_labels(df:DataFrame, start:int, stop:int)->DataFrame:
    expression = (f"case when time between {start} and {stop} then 'I'"
                  f"else 'NI' end")
    return df.withColumn('label', expr(expression))

In [77]:
def pipeline(info:dict):
    freq = int(info['sfreq'])
    start = int(info['seizure_start_sec'] * freq)
    stop = int(info['seizure_end_sec'] * freq)
    name = info['name'].replace('edf','parquet')
    
    df = spark.read.parquet(f"{path_data}{name}")
    df = set_labels(df=df, start=start, stop=stop)
    df.write.mode("overwrite").parquet(f"{path_data}{name.replace('.parquet','')}")
    

In [80]:
for index, row in infos.iterrows():
    pipeline(dict(row))